In [ ]:
select manual_labeling_matching.is_match as manual_match, prompt_matching.is_match as prompt_match 
from manual_labeling_matching
inner join prompt_matching on (
    prompt_matching.source_id = manual_labeling_matching.source_id and prompt_matching.target_id = manual_labeling_matching.target_id
)

In [1]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score

# Load the CSV file into a DataFrame
file_path = "data/sentence-result-kappa.csv"  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Ensure the column names match those in your CSV file
manual_a_labels = data['manual_match']
manual_b_labels = data['prompt_match']

# Compute Cohen's Kappa score
kappa = cohen_kappa_score(manual_a_labels, manual_b_labels)

print(f"Cohen's Kappa Score: {kappa}")

Cohen's Kappa Score: 0.3694948388572695


In [9]:
import psycopg2

pg_host = "localhost"
pg_database = "graph"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [19]:
import pandas as pd

pg_cursor.execute("""
with ctx_result as (
 	select * from result_option_llm where "method" = 'EUCLIDEAN'
) 
select ctx_result.is_similar as manual_match, result_option_llm_prompt.is_similar as prompt_match 
from ctx_result
inner join result_option_llm_prompt on (
    result_option_llm_prompt.source_id = ctx_result.source_id and result_option_llm_prompt.option_id = ctx_result.option_id
)
""")
result_sentence = pg_cursor.fetchall()
df_llm = pd.DataFrame(result_sentence, columns=['manual_match', 'prompt_match'])

In [20]:
pg_cursor.execute("""
with ctx_result as (
 	select * from result_option_graph where "method" = 'COSINE'
) 
select ctx_result.is_similar as manual_match, result_option_graph_prompt.is_similar as prompt_match 
from ctx_result
inner join result_option_graph_prompt on (
    result_option_graph_prompt.source_id = ctx_result.source_id and result_option_graph_prompt.option_id = ctx_result.option_id
)
""")
result_sentence = pg_cursor.fetchall()
df_graph = pd.DataFrame(result_sentence, columns=['manual_match', 'prompt_match'])

In [21]:
from sklearn.metrics import cohen_kappa_score

# manual_a_labels = df_graph['manual_match']
manual_a_labels = pd.concat([df_llm['manual_match'], df_graph['manual_match']]).tolist()
# manual_b_labels = df_graph['prompt_match']
manual_b_labels = pd.concat([df_llm['prompt_match'], df_graph['prompt_match']]).tolist()

# Compute Cohen's Kappa score
kappa = cohen_kappa_score(manual_a_labels, manual_b_labels)

print(f"Cohen's Kappa Score: {kappa}")

Cohen's Kappa Score: 0.5330494690397847
